# 将脚本作为管道作业运行

使用管道可将多个步骤分组到一个工作流中。 可使用组件生成管道。 每个组件都反映要运行的 Python 脚本。 组件在 YAML 文件中定义，该文件指定脚本及其运行方式。 

## 准备工作

需要最新版本的“azureml-ai-ml”包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装“azure-ai-ml”包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找必要的参数，请单击工作区右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 创建脚本

通过两个步骤生成管道：

1. 准备数据：修复缺失数据并规范化数据。
1. 训练模型：训练决策树分类模型。

运行以下单元以创建“src”文件夹和两个脚本。

In [ ]:
## 定义组件

若要定义组件，需要指定：

- 元数据：名称、显示名称、版本、说明、类型等。元数据可帮助描述和管理组件    。
- 接口：输入和输出 。 例如，模型训练组件将训练数据和正则化率作为输入，并生成训练的模型文件作为输出。 
- 命令、代码和环境：用于运行组件的命令、代码和环境  。 命令是用于执行组件的 shell 命令。 代码通常指源代码目录。 环境可以是 AzureML 环境（特选或自定义创建）、docker 映像或 conda 环境。

运行以下单元，为要作为管道步骤运行的每个组件创建 YAML。

In [ ]:
## 加载组件

定义每个组件后，可通过引用 YAML 文件来加载组件。 

## 构建管道

创建并加载组件后，可生成管道。 你将这两个组件组合到一个管道中。 首先，需要运行 `prep_data` 组件。 第一个组件的输出应该是第二个组件 `train_decision_tree` 的输入，第二个组件将训练模型。

`diabetes_classification` 函数表示完整的管道。 该函数需要一个输入参数：`pipeline_job_input`。 在设置过程中创建了一个数据资产。 你将使用已注册的数据资产作为管道输入。 

In [ ]:
可通过打印 `pipeline_job` 对象来检索管道作业的配置：

In [ ]:
可通过引用参数并指定新值来更改管道作业配置的任何参数：

In [ ]:
## 提交管道作业

最后，生成管道并将管道作业配置为根据需要运行后，可提交管道作业：

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job